In [ ]:
# 3.2 读取境外农场数据
import pymysql

# 建立连接对象
connection = pymysql.connect(
              host = '172.17.6.26',    
              port = 3306,      
              user = 'raa_user',    
              password = 'bigdata123',  
              db = 'OVERSEAS_FARM'       
                          )

try:
    with connection.cursor() as cursor: 
        #  查询import_farm表中的数据
        sql1 = "SELECT * FROM import_farm"
        cursor.execute(sql1)
        import_farm = cursor.fetchall() 

        # 查询import_nation表中的数据
        sql2 = "SELECT *  FROM import_nation"
        cursor.execute(sql2)
        import_nation = cursor.fetchall() 

finally:
    connection.close()

# 查看两表的第一行数据
print('import_farm的第一行数据:',import_farm[0])
print('import_nation的第一行数据:',import_nation[0])

In [ ]:
# 3.3 转换数据格式
import pandas as pd

# 将数据库导出的数据转换为DataFrame
import_farm = pd.DataFrame(list(import_farm),columns=['import_id','farm_id'])
import_nation = pd.DataFrame(list(import_nation),columns=['import_id','nation'])

# 查看转换后数据的前五行
print(import_farm.head())
print(import_nation.head())

In [ ]:
# 3.4 计算农场个数和进口次数
# 统计农场数量
num_farm = len(import_farm['farm_id'].unique())
print(num_farm)

# 统计进口次数
num_import = len(import_farm['import_id'].unique())
print(num_import)

In [ ]:
# 3.5 进口动物来源国家
# 获取进口动物的来源国家
nation_name = import_nation['nation'].unique()
print(nation_name)

#国家名称为英文的值替换为中文，替换结果在原数据上生效
import_nation['nation'].replace(['AUSTRALIA','Canada','New Zealand'], ['澳大利亚','加拿大','新西兰'], inplace=True)

In [ ]:
# 3.6 动物来源国的出口次数
# 查看重复记录的数量
num = import_nation.duplicated().sum()
print("重复记录的数量：\n", num)

# 删除重复记录，并在原数据上生效
import_nation.drop_duplicates(inplace = True)

# 查看nation列不同值个数
nation_count = import_nation['nation'].value_counts()

# 输出结果
print(nation_count)

In [ ]:
# 3.7 动物来源国家热力图可视化
from pyecharts.charts import Map

# 创建类实例
world_map = Map()

# 设置地图类型为世界地图
maptype = "world"

# 设置必要参数
world_map.add("出口动物次数",data, maptype, is_map_symbol_show=False)  # data = nation_count

# 不显示国家名称
world_map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

# 设置图标题
world_map.set_global_opts(
     title_opts=opts.TitleOpts(title="动物来源国家热力图"),
     visualmap_opts=opts.VisualMapOpts(max_=60,is_piecewise=True),
 )

In [ ]:
# 3.8 读取和转换境外检疫数据
import pymysql
import pandas as pd

# 建立连接对象
connection = pymysql.connect(
              host = '172.17.6.26',     
              port = 3306,      
              user = 'raa_user',     
              password = 'bigdata123', 
              db = 'OVERSEAS_QUARANTINE'       
                          )

try:
    with connection.cursor() as cursor:
        # 查询overseas_quarantine_record表的数据
        sql1 = "SELECT * FROM overseas_quarantine_record"
        cursor.execute(sql1)
        overseas_quarantine_record = cursor.fetchall() 

        # 查询overseas_lab_record表的数据
        sql2 = "SELECT * FROM overseas_lab_record"
        cursor.execute(sql2)
        overseas_lab_record = cursor.fetchall() 
finally:
    connection.close()

# overseas_quarantine_record表的列名
overseas_quarantine_col = ['import_id','overseas_field_id','prevention_disease','prevention_medicine','prevention_method','desinsectization_medicine','desinsectization_dosage','desinsectization_method','immunization_category','immunization_disease','immunization_method']

# overseas_lab_record表的列名
overseas_lab_col = ['import_id','overseas_lab','retention_method']

# 将数据转为DataFrame
overseas_quarantine_record = pd.DataFrame(list(overseas_quarantine_record),columns=overseas_quarantine_col)
overseas_lab_record = pd.DataFrame(list(overseas_lab_record),columns=overseas_lab_col)

# 查看数据的行列数
print('overseas_quarantine_record的行列数:',overseas_quarantine_record.shape)
print('overseas_lab_record的行列数:',overseas_lab_record.shape)

In [ ]:
# 3.9 获取境外检疫实验室名单
import pandas as pd

# 建立字典
replace_value = {
    'AsureQuality Limited Laboratory Services-Wellington Laboratory':'AsureQuality Ltd-Wellington Lab',
    'División Laboratorios Veterinarios (DILAVE)':'División Laboratorios Veterinarios',
    'Gribbles Veterinary Pathology Palmerston North':'Gribbles Veterinary Pathology',
    'Livestock Improvement Corporation (LIC) - Animal Health Laboratory':'Livestock Improvement Corporation (LIC)',    
}

# 替换
overseas_lab_record['overseas_lab'].replace(replace_value,inplace=True)

# 查看处理后的overseas_lab
lab_list = overseas_lab_record['overseas_lab'].unique()

# 输出结果
for lab in lab_list :
   print(lab)


In [ ]:
# 3.11 缺失值处理
# 统计overseas_start_record每列的缺失值数量
process_before = overseas_start_record.isnull().sum()
print("处理前：\n", process_before)

# 将所有缺失值处填充为"未知"
overseas_start_record.fillna('未知',inplace = True)


In [ ]:
# 3.13 运输工具可视化分析
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# 创建绘图对象
fig = plt.figure(figsize=(7,5))

# 统计不同运输工具的频数并绘制饼状图
transport_record['vehicle'].value_counts().plot.pie(autopct='%1.1f%%',colors=['cadetblue','yellowgreen'] ,fontsize=15)

# 设置图标题为`进口动物运输工具饼状图`
plt.title('进口动物运输工具饼状图',fontsize=15)

# 设置y轴名称为空，否则将默认显示列名
plt.ylabel('')

# 显示图形
plt.show()

In [ ]:
# 3.15 国内港口和机场位置分布图
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType

maptype = "china"  # 地图设置为中国
legend_harbor = "港口" # 设置港口图例名
legend_airport = "机场"  # 设置机场图例名
type_name = ChartType.EFFECT_SCATTER

city_map = Geo()

city_map.add_schema(
    maptype=maptype,
    itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="gray"),
    is_roam=True,
    zoom=1,
)

city_map.add(legend_harbor,harbor,type_=type_name,color="gold")
city_map.add(legend_airport,airport,type_=type_name,color="deepskyblue")

city_map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
city_map.set_global_opts(title_opts=opts.TitleOpts(title="抵达港口和机场位置分布图"))


city_map.render_notebook()

In [ ]:
# 3.16 国内城市接受进口动物次数热力图
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType

city_map = Geo()

maptype = 'china'
type_name = ChartType.EFFECT_SCATTER

# 设置为中国地图
city_map.add_schema(
    maptype=maptype,
    itemstyle_opts=opts.ItemStyleOpts(color="#323c48", border_color="gray"),
    is_roam=True,
    zoom=1,
)
# 设置热力图类型为散点 
city_map.add("抵达地点", data, type_=type_name, color="gold")

city_map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
city_map.set_global_opts(
     title_opts=opts.TitleOpts(title="国内城市接收进口动物次数热力图"),
     visualmap_opts=opts.VisualMapOpts(max_=30),)

city_map.render_notebook()

In [ ]:
# 3.18 每批动物死亡数量
# 需要替换的数据
replace_value = {
       '无':0,
       '6头':6,
       '死亡17头':17,
       '18头':18,
       '1头':1,
       '运输途中死32头':32,     
}
# 进行替换
domestic_arrive_record['death'].replace(replace_value,inplace = True)

# 取进口编号import_id和死亡数量death保存为data
data = domestic_arrive_record[['import_id','death']]
# 输出查看
print(data)

In [ ]:
# 3.20 各境内检疫场处理进口批次统计
import pandas as pd

# 按照domestic_field_id对domestic_quarantine_record分组
field_group = domestic_quarantine_record.groupby('domestic_field_id')

# 统计境内检疫场个数
n = len(field_group)
print('总共有%d个国内检疫场'%(n))

# 计算每个境内检疫场负责的进口批次
count = field_group['import_id'].unique()
print(count)

In [ ]:
# 3.22 进口动物数量统计
import matplotlib.pyplot as plt

#查看动物总数
num = animal_quarantine_result.shape[0]
print('动物总数为：%d'%(num))

#使用value_counts()查看category的取值情况
count = animal_quarantine_result['category'].value_counts()
print(count)

#绘图
fig = plt.figure(figsize=(8,6))
count.plot.bar(rot=0 ,colors=['cadetblue','yellowgreen','red','blue'], fontsize=15)
plt.title('各品种动物数量',fontsize=15)
plt.show()

In [ ]:
# 3.23 每一批进口动物的品种
# 按照import_id列对animal_quarantine_result分组
import_group = animal_quarantine_result.groupby('import_id')

# 保存结果
species = import_group['category'].unique()
print(species)

In [ ]:
# 3.25 建立农场黑名单
import pandas as pd

# 建立农场黑名单列表
farm_blacklist = farm_disease_rate[farm_disease_rate>0.5].index.tolist()
# 查看结果
print(farm_blacklist)

# 第四章

In [ ]:
# 4.2 连接动物来源国家表和动物检疫结果表
import pymysql
import pandas as pd

# 建立连接对象
connection = pymysql.connect(
             host = '172.17.6.26', 
             port = 3306, 
             user = 'raa_user',
             password = 'bigdata123',
    									 )                           
try:
    with connection.cursor() as cursor:
        sql = """
            SELECT animal_id,animal_quarantine_result.import_id,status,category,nation
            FROM OVERSEAS_FARM_PROCESSED.import_nation
            LEFT JOIN FINAL_RESULT_PROCESSED.animal_quarantine_result
            ON import_nation.import_id = animal_quarantine_result.import_id
             """
        cursor.execute(sql)
        data = cursor.fetchall() 
finally:
    connection.close()

# 将数据转为DataFrame
columns_name = ['animal_id','import_id','status','category','nation']
animal_nation = pd.DataFrame(list(data),columns=columns_name)

# 查看数据的大小
print('animal_nation的大小:',animal_nation.shape)

In [ ]:
# 4.3 计算动物来源国家的疫病率
# 分组
animal_nation_group = animal_nation.groupby('nation')

# 计算各国家疫病率
nation_disease_rate = round(animal_nation_group['status'].sum()/animal_nation_group['status'].count(),2)

# 输出结果
print(nation_disease_rate)

In [ ]:
# 4.4 进口动物品种的来源国家分布
# 分组
category_group = animal_nation.groupby('category')

# 获取每个动物品种的来源国家
category_nation = category_group['nation'].unique()

# 输出结果
print(category_nation)

In [ ]:
# 4.5  连接运输记录表和动物检疫结果表
import pymysql
import pandas as pd

# 建立连接对象
connection = pymysql.connect(
             host = '172.17.6.26',
             port = 3306, 
             user = 'raa_user',
             password = 'bigdata123',
    									 )                          
try:
    with connection.cursor() as cursor:
        sql = """
            SELECT animal_id, transport_record.import_id, status, category, start_location, arrive_location FROM 
            FINAL_RESULT_PROCESSED.animal_quarantine_result
            LEFT JOIN TRANSPORT_PROCESSED.transport_record 
            ON transport_record.import_id = animal_quarantine_result.import_id
            """
        cursor.execute(sql)
        data = cursor.fetchall() 
    
finally:
    connection.close()

# 将数据转为DataFrame
columns_name = ['animal_id', 'import_id', 'status', 'category', 'start_location', 'arrive_location']
animal_transport = pd.DataFrame(list(data), columns=columns_name)

# 查看数据的前5行
print(animal_transport.head())

In [ ]:
# 4.6 起运地点疫病率分析
import pandas as pd

# 对合并后的数据按起运地点进行分组
start_location_group = animal_transport.groupby('start_location')

# 计算每个起运地点的疫病率
start_location_rate = start_location_group['status'].sum()/start_location_group['status'].count()

# 显示结果
print(start_location_rate)

In [ ]:
# 4.7 抵达地点疫病率分析
import pandas as pd

# 对合并后的数据按抵达地点进行分组
arrive_location_group = animal_transport.groupby('arrive_location')

# 计算每个抵达地点疫病率
arrive_location_rate = arrive_location_group['status'].sum()/arrive_location_group['status'].count()

# 显示结果
print(arrive_location_rate)


In [ ]:
# 4.8 各抵达地点接受的动物品种
import pandas as pd

# 获取各抵达地点接收的动物品种
arrive_location_category = arrive_location_group['category'].unique()

# 显示结果
print(arrive_location_category)

In [ ]:
# 4.9 境外检疫分析的多表连接
import pymysql
import pandas as pd

# 建立连接对象
connection = pymysql.connect(
             host = '172.17.6.26',
             port = 3306, 
             user = 'raa_user',
             password = 'bigdata123',
    									 )                           
try:
    with connection.cursor() as cursor:
        sql = """
        SELECT animal_quarantine_result.animal_id,animal_quarantine_result.import_id,status,overseas_field_id,retention_method
        FROM FINAL_RESULT_PROCESSED.animal_quarantine_result,OVERSEAS_QUARANTINE_PROCESSED.overseas_quarantine_record,OVERSEAS_QUARANTINE_PROCESSED.overseas_lab_record
        WHERE 
        overseas_quarantine_record.import_id= animal_quarantine_result.import_id 
        AND  
        overseas_lab_record.import_id= animal_quarantine_result.import_id 
        AND 
        overseas_quarantine_record.import_id= overseas_lab_record.import_id
            """
        cursor.execute(sql)
        data = cursor.fetchall() 
finally:
    connection.close()

# 将数据转为DataFrame
columns_name = ['animal_id','import_id','status','overseas_field_id','retention_method']
animal_overseas_quarantine = pd.DataFrame(list(data),columns=columns_name)

# 查看数据的前5行
print(animal_overseas_quarantine.head())


In [ ]:
# 4.10 境外检疫场的疫病情况
import pandas as pd

# 分组
field_group = animal_overseas_quarantine.groupby('overseas_field_id')

# 计算各境外检疫场的疫病率
overseas_field_rate = field_group['status'].sum()/field_group['status'].count()

# 显示结果
print(overseas_field_rate)

In [ ]:
# 4.11 不同样品保存方法的疫病率
import pandas as pd

# 分组
retention_method_group = animal_overseas_quarantine.groupby('retention_method')

# 计算不同样品保存方法的疫病率
retention_method_rate = retention_method_group['status'].sum()/retention_method_group['status'].count()

# 显示结果
print(retention_method_rate)

# 第五章

In [ ]:
# 5.2 确定建模动物品种
import pandas as pd

# 按照动物品种进行分组
category_group = animal_quarantine_result.groupby('category')

# 计算各品种疫病率
category_disease_rate = category_group['status'].sum()/category_group['status'].count()

# 显示结果
print(category_disease_rate)


In [ ]:
# 5.3 筛选目标动物品种数据
import pandas as pd

# 筛选出牛的数据
cow = animal_quarantine_result[animal_quarantine_result['category'] == '牛']
print('牛的数据集大小：',cow.shape)

In [ ]:
# 5.4 确定目标疫病
import pandas as pd

cow_disease_list = ["牛传染性鼻气管炎","牛病毒性腹泻","赤羽病","副结核病","鹿流行性出血热","阳性结果超限淘汰","结核病","牛地方流行性白血病","蓝舌病","规定疫病种类全群淘汰","布氏杆菌","边界病","衣原体病",]

cow_disease_data = {}

for disease in cow_disease_list:         # 遍历每一种病
    label = []
    for disease_data in cow['disease_name']:  # 遍历每一只动物
        if disease in disease_data:    # 判断该病是否存在于某只动物上
            label.append(1)
        else:
            label.append(0)
    cow_disease_data[disease] = label       # 存储每个疫病的数据

# 生成DataFrame
cow_disease_data = pd.DataFrame(cow_disease_data)

# 每种疫病患病数量的降序排序
answer = cow_disease_data.sum().sort_values(ascending=False)

print(answer)

In [ ]:
# 5.5 提取目标疫病标签数据
import pandas as pd

# 提取牛传染性鼻气管炎标签数据
disease_IBR = cow_disease_data["牛传染性鼻气管炎"]

# 提取牛传染性鼻气管炎标签数据
disease_EBL = cow_disease_data["牛地方流行性白血病"]

# 输出数据大小
print(disease_IBR.shape)
print(disease_EBL.shape)

In [ ]:
# 5.6 利用农场疫病率替换农场名称
import pandas as pd

# 连接数据
cow = pd.merge(cow,farm_disease_rate,how='left',on='farm_id')

# 查看连接结果
print(cow.info())

In [ ]:
# 5.8 特征独热编码
import pandas as pd

# 独热编码
cow_one_hot = pd.get_dummies(cow,columns=None)

# 查看结果
print(cow_one_hot.shape)

In [ ]:
# 5.9 训练集测试集划分
import pandas as pd
from sklearn.model_selection import train_test_split

# 使用train_test_split方法，将x,y划分训练集和测试集
x_train,x_test,y_train,y_test = train_test_split(cow_one_hot,disease_IBR,test_size=0.2, random_state=30)

# 查看训练集和测试集的占比
print('训练集占比: ',round(len(x_train)/len(cow_one_hot),2))
print('测试集占比: ',round(len(x_test)/len(cow_one_hot),2))

In [ ]:
# 5.10 对牛传染性鼻气管炎建立逻辑回归模型
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score

# 建立逻辑回归模型
lgr = LogisticRegression()

# 训练模型
lgr.fit(x_train, y_train)

# 模型预测
pred_test = lgr.predict(x_test)
        
# 计算召回率
recall_disease_IBR  = round(recall_score(y_true=y_test,y_pred=pred_test),3)

# 输出召回率
print(recall_disease_IBR)   

In [ ]:
# 5.11 使用随机森林对牛传染性鼻气管炎建模
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import recall_score

# 建立随机森林模型
rf = RandomForestClassifier(random_state=30)

# 训练模型
rf.fit(x_train,y_train)

# 模型预测
pred_test = rf.predict(x_test)
        
# 计算召回率
recall_disease_IBR = round(recall_score(y_true=y_test,y_pred=pred_test),3)

# 输出召回率
print(recall_disease_IBR) 

In [ ]:
# 5.12 过采样处理
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score

# 过采样处理
somte = SMOTE(ratio=1,random_state=1) 
x_train_smote, y_train_smote = somte.fit_sample(x_train,y_train)

# 建立随机森林模型
rf = RandomForestClassifier(random_state=30)

# 过采样后建模
rf.fit(x_train,y_train)

# 使用训练好的模型对测试集进行预测
pred_test = rf.predict(x_test)
        
# 使用召回率查看模型效果
recall_disease_IBR = round(recall_score(y_true=y_test, y_pred=pred_test),3)
# 输出模型召回率
print(recall_disease_IBR)   

In [ ]:
# 5.13 参数调优
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 数据集划分
x_train,x_test,y_train,y_test = train_test_split(cow_one_hot,disease_IBR,test_size=0.2,random_state=30)

# 过采样处理
sm = SMOTE(ratio=1,random_state=1) 
x_train_smote,y_train_smote = sm.fit_sample(x_train,y_train)

# 建立模型
rf = RandomForestClassifier(n_estimators=25,max_depth=10,random_state=30)

# 训练模型
rf.fit(x_train_smote,y_train_smote)

# 预测
pred_test = rf.predict(x_test)

# 计算召回率
recall_disease_IBR = round(recall_score(y_true=y_test,y_pred=pred_test),3)
print(recall_disease_IBR)

In [ ]:
# 5.14 牛地方流行性白血病建模的过采样处理
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# 划分数据集
x_train,x_test,y_train,y_test = train_test_split(cow_one_hot,disease_EBL,test_size=0.2,random_state=30)

# 过采样处理
somte = SMOTE(ratio=1,random_state=1) 
x_train_smote, y_train_smote = somte.fit_sample(x_train,y_train)

In [ ]:
# 5.15 使用逻辑回归对牛地方流行性白血病构建模型
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score

# 建立逻辑回归模型
lgr = LogisticRegression()

# 过采样后建模
lgr.fit(x_train, y_train)

# 使用训练好的模型对测试集进行预测
pred_test = lgr.predict(x_test)
        
# 使用召回率查看模型效果
recall_disease_EBL = round(recall_score(y_true=y_test, y_pred=pred_test),3)

# 输出模型召回率
print(recall_disease_EBL)   

In [ ]:
# 6.2 牛传染性鼻气管炎检测模型的混淆矩阵
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
import seaborn as sns
import matplotlib.pyplot as plt

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_test,pred_test)

# 绘制混淆矩阵热力图
fig,ax = plt.subplots(figsize=(5, 4))
sns.heatmap(conf_matrix, ax=ax, annot=True, annot_kws={'size':15},  fmt='d')
ax.set_ylabel('真实标签')
ax.set_xlabel('预测标签')
ax.set_title('混淆矩阵热力图')
plt.show()

# 输出召回率
recall_disease_IBR =  round(recall_score(y_true=y_test, y_pred=pred_test),3)
print('牛传染性鼻气管炎检测模型的召回率为：',recall_disease_IBR)

In [ ]:
# 6.3 牛传染性鼻气管炎检测模型的ROC曲线
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# 计算AUC值
auc = roc_auc_score(y_test,prob).round(3)
print('牛传染性鼻气管炎检测模型的AUC值为：', auc)

# 计算ROC曲线需要的数据
fpr, tpr, threshold = roc_curve(y_test, prob)

# 绘制ROC曲线
fig = plt.figure(figsize=(5,4), facecolor='white')
plt.plot(fpr, tpr,  color='teal')  #设置横轴和纵轴数据
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.tick_params(labelsize=15)
plt.xlabel('FPR', fontsize=15)
plt.ylabel('TPR', fontsize=15)
plt.title('牛传染性鼻气管炎检测模型的ROC曲线')
plt.show()

In [ ]:
# 6.5 牛地方流行性白血病检测模型的混淆矩阵
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
import seaborn as sns
import matplotlib.pyplot as plt

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_test,pred_test)

# 绘图
fig,ax = plt.subplots(figsize=(5,4))
sns.heatmap(conf_matrix, ax=ax, annot=True, annot_kws={'size':15}, fmt='d')
ax.set_ylabel('真实标签')
ax.set_xlabel('预测标签')
ax.set_title('混淆矩阵热力图')
plt.show()

# 输出召回率
recall_disease_EBL = round(recall_score(y_true=y_test, y_pred=pred_test), 3)
print('牛地方流行性白血病检测模型的召回率为：', recall_disease_EBL)

In [ ]:
# 6.6 牛地方流行性白血病检测模型的ROC曲线
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# 计算AUC值
auc = roc_auc_score(y_test,prob)
print('牛地方流行性白血病检测模型的AUC值为：', auc)

# 计算ROC曲线需要的数据
fpr, tpr, threshold = roc_curve(y_test,prob)

# 绘制ROC曲线
fig = plt.figure(figsize=(5, 4), facecolor='white')
plt.plot(fpr, tpr,  color='teal')  
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.tick_params(labelsize=15)
plt.xlabel('FPR', fontsize=15)
plt.ylabel('TPR', fontsize=15)
plt.title('牛地方流行性白血病检测模型的ROC曲线')
plt.show()
